In [1]:
"""
Just a demo of using direct calls with OpenAI util to parse and use tools
In this example there simple numeric operations
"""

'\nJust a demo of using direct calls with OpenAI util to parse and use tools\nIn this example there simple numeric operations\n'

In [2]:
import os
import openai
from openai import OpenAI
import json

import os
from dotenv import load_dotenv

In [ ]:


api_key = os.getenv("OPENAI_API_KEY")


# Define tools 

In [ ]:


tools = [
    {
        "type": "function",
        "function": {
            "name": "add_numbers",
            "description": "Add two numbers",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "First number"},
                    "b": {"type": "number", "description": "Second number"},
                },
                "required": ["a", "b"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "multiply_numbers",
            "description": "Multiply two numbers",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "First number"},
                    "b": {"type": "number", "description": "Second number"},
                },
                "required": ["a", "b"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "subtract_numbers",
            "description": "Subtract second number from the first",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "First number"},
                    "b": {"type": "number", "description": "Second number"},
                },
                "required": ["a", "b"],
            },
        },
    }
]

# Define actual implementations of the tools
def add_numbers(a, b):
    return {"result": a + b}

def multiply_numbers(a, b):
    return {"result": a * b}

def subtract_numbers(a, b):
    return {"result": a - b}

tool_functions = {
    "add_numbers": add_numbers,
    "multiply_numbers": multiply_numbers,
    "subtract_numbers": subtract_numbers,
}




# Define tool handlers 

In [105]:
def call_function(func_name : str, args : dict):
    func_obj = tool_functions[func_name]
    return func_obj(**args)
    
def handle_tool_call(tool_call):


    tool_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    print(f"Tool called: {tool_name}", flush=True)

    result = call_function(tool_name, arguments)

    res = {"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id}

    return res

def handle_multi_tool_calls(tool_calls):
    res = []
    for tool_call in tool_calls:
        tool_result = handle_tool_call(tool_call)
        res.append(tool_result)
    return res

# One off tool use

In [77]:
# Define prompts 

system_prompt = (
 "You are a helpful AI assistant. "
  "If a user asks a question that requires calculation, "
  "use the available functions to compute the answer. "
  "Do not make up results. "
)

user_prompt = "What is product of 5 and 7?"

input_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
    ]


In [78]:
# define model / client
#-------------------------------------------------------------
client = OpenAI(api_key=api_key)
client_local = OpenAI(base_url="http://localhost:1234/v1", api_key="no_key")

model_name = "gpt-4o"
model_name_local = "google/gemma-3-12b"


# run llm 
#-------------------------------------------------------------
client_use = client
model_name_use = model_name

response1 = client_use.chat.completions.create(
    model= model_name_use, 
     messages=input_messages, 
     tools=tools
)

finish_reason = response1.choices[0].finish_reason


# call tool (ONLY ONE TOOL)
#-------------------------------------------------------------
if finish_reason == "tool_calls":

    message = response1.choices[0].message
    
    tool_call = message.tool_calls[0] # take first! 
    tool_result = handle_tool_call(tool_call)
    
    # update unput messages 
    new_input_messages = input_messages.copy()
    new_input_messages.append(message)
    new_input_messages.extend([tool_result])

    # run llm again 
    #-------------------------------------------------------------
    response2 = client_use.chat.completions.create(
        model=model_name_use, 
        messages=new_input_messages, 
        tools=tools
    )

    print(response2.choices[0].message.content)

else:
    print(response1.choices[0].message.content)




Tool called: multiply_numbers
The product of 5 and 7 is 35.


# Multiple tools use (cascade)

In [102]:

# Define prompts 

system_prompt = (
 "You are a helpful AI assistant. "
  "If a user asks a question that requires calculation, "
  "use the available functions to compute the answer. "
  "Do not make up results. "
  "You may call multiple functions if needed."
)

user_prompt = "What is product of 5 and 7 minus 20 times 2?"

input_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
    ]

In [103]:
# Step 1: just call api 

# define model / client
#-------------------------------------------------------------
client = OpenAI(api_key=api_key)
client_local = OpenAI(base_url="http://localhost:1234/v1", api_key="no_key")

model_name = "gpt-4o"
model_name_local = "google/gemma-3-12b"


# run llm 
#-------------------------------------------------------------
client_use = client
model_name_use = model_name

response1 = client_use.chat.completions.create(
    model= model_name_use, 
     messages=input_messages, 
     tools=tools
)

finish_reason = response1.choices[0].finish_reason

In [104]:
# Step 2: while there are tools keep calling 
done_status = False
max_iters = 5
cur_iter = 1

response_rolling = response1

new_input_messages = input_messages.copy()

while not done_status: 

    if cur_iter > max_iters:
        break
    
    # invoke LLM only on the second iteration (first one is for processing tools from step 1)
    if cur_iter > 1:
        response_rolling = client_use.chat.completions.create(
                            model= model_name_use, 
                            messages=new_input_messages, 
                            tools=tools
                        )
        # update finish reason 
        finish_reason = response_rolling.choices[0].finish_reason
    
    if finish_reason == "tool_calls":

        message = response_rolling.choices[0].message
        tool_calls = message.tool_calls

        res_tools = handle_multi_tool_calls(tool_calls)

        new_input_messages.append(message)
        new_input_messages.extend(res_tools)
        
    else:
        done_status = True


    cur_iter += 1

print(response_rolling.choices[0].message.content)

Tool called: multiply_numbers
Tool called: multiply_numbers
Tool called: subtract_numbers
The product of 5 and 7 minus 20 times 2 is \(-5\).
